In [ ]:
import os
import torchvision
import torchvision.transforms as transforms

if not os.path.exists("data"):
    os.makedirs("data")

cifar10_train = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
cifar10_test = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
print("✅ CIFAR10 disponible en carpeta 'data/'")

# Entrenamiento con VGG16

Este notebook entrena **VGG16** y sus variantes sobre el dataset CIFAR10.

In [ ]:
import torch
import torch.nn as nn
from utils import create_data_loaders, train_model, test_model, count_parameters
from models import VGG16, VGG16_BN, VGG16_GAP
from config import TRAINING_CONFIG, DATA_CONFIG, PATHS
import os

In [ ]:
# Preparar los dataloaders
train_loader, val_loader, test_loader, class_names = create_data_loaders(
    DATA_CONFIG['data_dir'],
    batch_size=TRAINING_CONFIG['batch_size'],
    input_size=DATA_CONFIG['input_size'],
    train_split=DATA_CONFIG['train_split'],
    val_split=DATA_CONFIG['val_split'],
    seed=DATA_CONFIG['seed']
)

In [ ]:
# Entrenar modelo base (VGG16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG16(nChannels=3, nClasses=len(class_names)).to(device)
history, avg_time = train_model(
    model, train_loader, val_loader,
    epochs=TRAINING_CONFIG['epochs'],
    optimizer_name=TRAINING_CONFIG['optimizer'],
    lr=TRAINING_CONFIG['learning_rate'],
    device=device,
    save_path=os.path.join(PATHS['checkpoints'], 'vgg16.pth')
)
print(f"Número de parámetros entrenables: {count_parameters(model)/1e6:.2f}M")

In [ ]:
# Evaluar modelo en test set
test_acc, test_f1, conf_matrix, report = test_model(model, test_loader, device, class_names)
print("\nResultados en Test:")
print(report)